In [ ]:
import sys
sys.path.append('../')
import torch
import numpy as np
import pandas as pd

from pathlib import Path
from BaseTester import BaseTester
from demo import u0, f0
from MyPlot import *
from utils import hard_encode
import torch.nn.functional as F

In [ ]:
from demo2 import DescentGenerator

batch_size = 3
GridSize = 64
dtype = torch.float
device = 'cpu'
maxiter = 5
area = ((0, 0), (1, 1))
generator = DescentGenerator(
    batch_size,
    GridSize,
    dtype,
    device,
    maxiter,
    area
    )

pre = torch.rand((batch_size, 1, GridSize-2, GridSize-2))
kappa = torch.rand((batch_size, 1, GridSize, GridSize))
force = torch.rand((batch_size, 1, GridSize, GridSize))

y = generator(pre, force, kappa)

In [ ]:
class ResidualLoss(torch.nn.Module):
	def __init__(self, dtype, device, h, loss_fn, gd=0):
		super().__init__()
		self.dtype = dtype
		self.device = device
		self.h = h
		self.loss_fn = loss_fn
		self.gd = gd

		self.k = self._get_kernel([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
		self.hard_encode = lambda x: hard_encode(x, self.gd)

	def _get_kernel(self, k):
		k = torch.tensor(k, requires_grad=False)
		k = k.view(1, 1, 3, 3).repeat(1, 1, 1, 1).to(self.dtype).to(self.device)
		return k
		
	def forward(self, pre, f, ans):
		u = self.hard_encode(pre)
		x = F.conv2d(u, self.k)
		y = F.conv2d(ans[None, None, ...], self.k) 
		return torch.abs(y.squeeze() - x.squeeze())
	
class Demo1Tester(BaseTester):
	def __init__(self, k, **kwargs):
		super().__init__(**kwargs)
		self.k = k
		self.mesh()
		self.force = f0(self.xx, self.yy, k)					
		self.ans = u0(self.xx, self.yy, k)

		self.data = torch.from_numpy(np.stack([self.xx, self.yy])).to(self.dtype).to(self.device)[None, ...]
		
	def l2(self, pre, ans):
		diff = (pre - ans) ** 2 * self.h**2
		l2_errors = np.sqrt(np.sum(diff))
		return l2_errors.mean() 
	
	def mesh(self):
		(self.left, self.bottom), (self.right, self.top) = self.area
		self.dx = (self.right - self.left) / (self.GridSize - 1)
		self.dy = (self.top - self.bottom) / (self.GridSize - 1)
		self.h = self.dx

		self.xx, self.yy = np.meshgrid(
			np.linspace(self.left, self.right, self.GridSize),
			np.linspace(self.bottom, self.top, self.GridSize),
		)
	
	def test(self, exp_names, best_or_last):
		df = {
			'exp_name': [],
			'l2': []
		}

		for exp_name in exp_names:
			self.load_kwargs(exp_name)
			self.load_ckpt(best_or_last, exp_name)

			with torch.no_grad():
				pre = self.net(self.data)
				pre = hard_encode(pre, 0).squeeze()
				pre = pre.cpu().numpy()

				ans = self.ans
				img_save_path = Path(f"{self.img_save_path}/{exp_name}")
				if not img_save_path.is_dir():
					img_save_path.mkdir(parents=True)
				self.save_img(f"{img_save_path}", pre, ans)

				l2_error = self.l2(pre.squeeze(), ans.squeeze())
				print(f"{self.k}\t&\t{self.kwargs['method']} \t&\t{self.kwargs['net_kwargs']['act']} \t&\t {self.kwargs['net_kwargs']['norm_method']} \t&\t{l2_error:.5e}")
				df['exp_name'].append(exp_name)
				df['l2'].append(l2_error)
		
		df = pd.DataFrame(df)
		df.to_csv(f"{self.img_save_path}/k={self.k}-l2.csv", index=False)
	
	def save_img(self, save_path, pre, ans):
		save_ctf(save_path, pre, ans, self.xx, self.yy)
		levels = np.linspace(self.ans.min(), self.ans.max(), 12)[2:-2]
		save_contour(save_path, pre, ans, self.xx, self.yy, levels=levels)
		save_surf(save_path, pre, self.xx, self.yy, 'surf_pre')
		save_surf(save_path, ans, self.xx, self.yy, 'surf_ans')
	

In [ ]:
GridSize = 256
k = 8
Tester = Demo1Tester(
    k = k,
	GridSize=GridSize,
	area=((-1, -1), (1, 1)),
	ckpt_save_path=f'model_save',
	hyper_parameters_save_path = f'hyper_parameters', 
	img_save_path = f'./images', 
	device='cuda',
	dtype=torch.float,
)
exp_names = []
for p in Path(f'model_save/{GridSize}').iterdir():
	if 'Energy' in p.stem or 'MatRes' in p.stem:
		continue
	if f'k={k}' in p.stem:
		exp_names.append(p.stem)
Tester.test(
	exp_names,
	'last'
)

In [ ]:
import sys
sys.path.append('../')
import torch
import numpy as np
import pandas as pd

from pathlib import Path
from BaseTester import BaseTester
from MyPlot import *
from utils import hard_encode
from utils import normal, reaction_A, reaction_b_dir
from scipy.sparse.linalg import spsolve

In [18]:


class Demo2Tester(BaseTester):
	def __init__(self, testN, **kwargs):
		super().__init__(**kwargs)
		self.testN = testN
		self.mesh()
		self.force =  normal(self.xx, self.yy, self.h, ((self.right - self.left)/2, (self.top-self.bottom) / 2))
		
	def l2(self, pre, ans):
		diff = (pre - ans) ** 2 * self.h**2
		l2_errors = np.sqrt(np.sum(diff))
		return l2_errors.mean() 
	
	def mesh(self):
		(self.left, self.bottom), (self.right, self.top) = self.area
		self.dx = (self.right - self.left) / (self.GridSize - 1)
		self.dy = (self.top - self.bottom) / (self.GridSize - 1)
		self.h = self.dx

		self.xx, self.yy = np.meshgrid(
			np.linspace(self.left, self.right, self.GridSize),
			np.linspace(self.bottom, self.top, self.GridSize),
		)
	
	def test(self, exp_names, best_or_last, kappas=None):
		if kappas is None:
			# kappas = np.random.rand(self.testN, 1, self.GridSize, self.GridSize)
			kappas = np.random.uniform(0.1, 2,  (self.testN, 1, self.GridSize, self.GridSize))
			# kappas = np.random.normal(0.5, 0.2, (self.testN, 1, self.GridSize, self.GridSize))
		ans = []
		b = reaction_b_dir(self.force, 0, self.h)
		testN = len(kappas)
		for i in range(testN):
			k = kappas[i].squeeze()
			A = reaction_A(self.GridSize, k).tocsr()
			ans.append(spsolve(A, b).reshape( self.GridSize, self.GridSize))
		ans = np.stack(ans)
		data = torch.from_numpy(kappas).to(self.dtype).to(self.device)
		
		for exp_name in exp_names:
			df = {
				'test_case': [],
				'l2': []
			}
		
			self.load_kwargs(exp_name)
			self.load_ckpt(best_or_last, exp_name)
		
			with torch.no_grad():
				pre = self.net(data)
				pre = hard_encode(pre, 0)
				pre = pre.cpu().numpy().squeeze()

			save_path = Path(f"{self.img_save_path}/{exp_name}")
			if not save_path.is_dir():	save_path.mkdir(parents=True)

			l2_errors = []
			for i in range(testN):
				p = save_path/f"Case-{i}"
				if not p.is_dir():	
					p.mkdir(parents=True)
				self.save_img(p, pre[i], ans[i], kappas[i].squeeze())
				l2_error = self.l2(pre[i], ans[i])
				l2_errors.append(l2_error)
				df['test_case'].append(i)
				df['l2'].append(l2_error)
			df = pd.DataFrame(df)
			df.to_csv(f"{save_path}/Demo2-l2.csv", index=False)

			method, k, trainN = exp_name.split('-')[-3:]
			print(f"{method}\t&\t{k}\t&\t{trainN}\t&\t{np.array(l2_errors).mean()}")
	
	def save_img(self, save_path, pre, ans, cof):
		save_img_force(save_path, cof, name='cof', vmin=0.1, vmax=2)
		save_ctf(save_path, pre, ans, self.xx, self.yy)
		levels = np.linspace(ans.min(), ans.max(), 20)[1:-11]
		save_contour(save_path, pre, ans, self.xx, self.yy, levels=levels)
		save_surf(save_path, pre, self.xx, self.yy, 'surf_pre')
		save_surf(save_path, ans, self.xx, self.yy, 'surf_ans')
	

In [19]:
GridSize = 256
Tester = Demo2Tester(
	testN=3,
	GridSize=GridSize,
	area=((0, 0), (1, 1)),
	ckpt_save_path=f'model_save',
	hyper_parameters_save_path = f'hyper_parameters', 
	img_save_path = f'./images/Demo2', 
	device='cuda',
	dtype=torch.float,
)
exp_names = []
for p in Path(f'model_save/{GridSize}').iterdir():
	if 'Demo2' in p.stem and '-25-' in p.stem and "-5000" in p.stem:
		exp_names.append(p.stem)
		# print(p.stem.split('-')[-3:])
Tester.test(
	exp_names,
	'last',
	# kappas=np.ones((2, 1, 256, 256)) * 0.2

)

Desc	&	25	&	5000	&	0.0008215464322893608
Jac	&	25	&	5000	&	0.0005291946548090439


In [ ]:
Jac		&	1	&	100		&	$9.21e^{-2}$
Jac		&	1	&	500		&	$7.93e^{-3}$
Jac		&	1	&	5000	&	$2.66e^{-3}$

Jac		&	5	&	100		&	$2.83e^{-2}$
Jac		&	5	&	500		&	$1.61e^{-3}$
Jac		&	5	&	5000	&	$2.76e^{-3}$

Jac		&	25	&	100		&	$2.12e^{-2}$
Jac		&	25	&	500		&	$1.26e^{-3}$
Jac		&	25	&	5000	&	$7.29e^{-4}$

Desc	&	1	&	100		&	$8.68e^{-2}$
Desc	&	1	&	500		&	$7.50e^{-3}$
Desc	&	1	&	5000	&	$4.52e^{-3}$

Desc	&	5	&	100		&	$6.50e^{-2}$
Desc	&	5	&	500		&	$3.49e^{-3}$
Desc	&	5	&	5000	&	$1.97e^{-3}$

Desc	&	25	&	100		&	$2.04e^{-2}$
Desc	&	25	&	500		&	$1.78e^{-3}$
Desc	&	25	&	5000	&	$6.67e^{-4}$

